# Tutorial Completo: Deep Q-Network (DQN) y Variantes

Exploraremos los fundamentos de aprendizaje por refuerzo profundo (Deep RL) con DQN y sus mejoras.

## Contenido:
1. Introducción a Deep RL (3 celdas)
2. Fundamentos de PyTorch (4 celdas)
3. Arquitectura DQN con CartPole (12 celdas)
4. Double DQN (10 celdas)
5. Dueling DQN (10 celdas)
6. Comparación de los 3 Algoritmos (8 celdas)
7. Ejercicios Prácticos (5 celdas)
8. Conclusiones (3 celdas)

## Sección 1: Introducción a Deep RL

### ¿Qué es Deep Reinforcement Learning?

Deep RL combina:
- **Aprendizaje por Refuerzo (RL)**: El agente aprende mediante interacción con el ambiente
- **Redes Neuronales Profundas**: Para aproximar funciones complejas (políticas, Q-values, etc.)

### Problema de Q-Learning Clásico
- Con espacios de estados grandes/continuos es impracticable usar una tabla Q
- Solución: Aproximar Q(s,a) con una red neuronal

### Deep Q-Network (DQN)
Paper seminal: "Human-level control through deep reinforcement learning" (Mnih et al., 2015)

**Componentes clave:**
1. **Experience Replay**: Almacena transiciones y entrena con mini-batches aleatorios
2. **Target Network**: Red separada para calcular targets, actualizada periódicamente

### Ecuación de Bellman para DQN

$$Q(s,a) \approx r + \gamma \max_{a'} Q(s', a')$$

**Ventajas de redes neuronales profundas:**
- Generalizan bien a nuevos estados
- Pueden aprender representaciones complejas
- Escalables a problemas de alta dimensión (imágenes, etc.)

**Desafíos:**
- Correlación temporal entre datos
- Bootstrapping (el target depende de la misma red)
- Sobreestimación de Q-values

### Timeline de Mejoras

| Año | Algoritmo | Mejora Principal |
|-----|-----------|------------------|
| 2015 | DQN | Experience Replay + Target Network |
| 2015 | Double DQN | Reducir sobreestimación de Q-values |
| 2016 | Dueling DQN | Separar Value y Advantage streams |
| 2017 | Rainbow | Combinar todos los anteriores + PER + Noisy Networks |

**Nuestro enfoque:** Estudiar DQN, Double DQN y Dueling DQN en profundidad

## Sección 2: Fundamentos de PyTorch para Deep RL

In [ ]:
# Importar librerías necesarias
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
from collections import deque, namedtuple
import random
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Verificar GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo disponible: {device}")
print(f"PyTorch version: {torch.__version__}")

### Conceptos Básicos de PyTorch para RL

1. **Tensores**: Equivalente a ndarrays de NumPy pero en GPU
2. **Autograd**: Cálculo automático de gradientes
3. **nn.Module**: Clase base para definir redes neuronales
4. **Optimizadores**: Adam, SGD, etc. para actualizar parámetros

In [ ]:
# Ejemplo: Red neuronal simple para Q-Network
class SimpleQNetwork(nn.Module):
    """Red simple para aproximar Q-values"""
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(SimpleQNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, state):
        """state -> Q-values para cada acción"""
        return self.net(state)

# Ejemplo de uso
state_dim = 4  # CartPole state dimension
action_dim = 2  # CartPole action dimension

q_net = SimpleQNetwork(state_dim, action_dim).to(device)
print("Arquitectura de la red:")
print(q_net)

# Forward pass
dummy_state = torch.randn(1, state_dim).to(device)
q_values = q_net(dummy_state)
print(f"\nEstado (batch=1): {dummy_state.shape}")
print(f"Q-values output: {q_values.shape}")
print(f"Q-values: {q_values}")

In [ ]:
# Ejemplo: Entrenamiento básico
optimizer = optim.Adam(q_net.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Simular un paso de entrenamiento
batch_states = torch.randn(32, state_dim).to(device)  # 32 estados
batch_actions = torch.randint(0, action_dim, (32,)).to(device)  # acciones
batch_targets = torch.randn(32).to(device)  # targets Q-values

# Forward pass
q_values = q_net(batch_states)
q_values_for_actions = q_values.gather(1, batch_actions.unsqueeze(1)).squeeze()

# Calcular pérdida
loss = loss_fn(q_values_for_actions, batch_targets)

# Backward pass
optimizer.zero_grad()
loss.backward()
torch.nn.utils.clip_grad_norm_(q_net.parameters(), 1.0)  # Clipping de gradientes
optimizer.step()

print(f"Loss: {loss.item():.4f}")
print("\nComponentes clave del entrenamiento:")
print("1. Forward pass: calcular Q(s,a)")
print("2. Loss: MSE entre Q actual y target")
print("3. Backward: calcular gradientes")
print("4. Optimization step: actualizar parámetros")

## Sección 3: Arquitectura DQN con CartPole

Implementaremos DQN paso a paso, entrenándolo en el ambiente CartPole-v1

### 3.1 Replay Buffer

Almacena transiciones (s, a, r, s', done) para romper correlación temporal

In [ ]:
# Definir Transition
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayBuffer:
    """Experience Replay Buffer"""
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append(Transition(state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    
    def __len__(self):
        return len(self.buffer)

# Prueba
replay_buffer = ReplayBuffer(capacity=1000)
for i in range(100):
    state = np.random.randn(4)
    action = 0
    reward = 1.0
    next_state = np.random.randn(4)
    done = False
    replay_buffer.push(state, action, reward, next_state, done)

print(f"Buffer size: {len(replay_buffer)}")
batch = replay_buffer.sample(10)
print(f"Sample batch size: {len(batch)}")
print(f"First transition: state shape {batch[0].state.shape}, action: {batch[0].action}")

### 3.2 Red DQN Básica

In [ ]:
class DQN(nn.Module):
    """Red Neuronal para Q-Network"""
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(DQN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# Crear instancia
q_network = DQN(state_dim=4, action_dim=2).to(device)
target_network = DQN(state_dim=4, action_dim=2).to(device)
target_network.load_state_dict(q_network.state_dict())
target_network.eval()  # Target network en modo evaluación

print("Q-Network:")
print(q_network)
print("\nTarget Network (copia de Q-Network):")
print("Los pesos se sincronizan cada N episodios")

### 3.3 Agente DQN

In [ ]:
class DQNAgent:
    """Agente DQN completo"""
    def __init__(self, state_dim, action_dim, 
                 learning_rate=1e-3,
                 gamma=0.99,
                 epsilon_start=1.0,
                 epsilon_end=0.01,
                 epsilon_decay=500,
                 buffer_size=10000,
                 batch_size=64,
                 target_update=10):
        
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update = target_update
        self.device = device
        
        # Redes
        self.q_network = DQN(state_dim, action_dim).to(self.device)
        self.target_network = DQN(state_dim, action_dim).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        # Optimizador y buffer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.replay_buffer = ReplayBuffer(buffer_size)
        self.steps = 0
    
    def get_action(self, state, training=True):
        """ε-greedy action selection"""
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_network(state_tensor)
            return q_values.argmax(dim=1).item()
    
    def update_epsilon(self):
        """Decay epsilon exponencialmente"""
        self.epsilon = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
                       np.exp(-self.steps / self.epsilon_decay)
    
    def store_transition(self, state, action, reward, next_state, done):
        self.replay_buffer.push(state, action, reward, next_state, done)
    
    def train_step(self):
        """Un paso de entrenamiento"""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # Muestrear batch
        transitions = self.replay_buffer.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        # Convertir a tensores
        state_batch = torch.FloatTensor(np.array(batch.state)).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch.next_state)).to(self.device)
        done_batch = torch.FloatTensor(batch.done).to(self.device)
        
        # Q-values actuales
        current_q = self.q_network(state_batch).gather(1, action_batch)
        
        # Targets: r + γ * max Q_target(s', a')
        with torch.no_grad():
            next_q = self.target_network(next_state_batch).max(1)[0]
            target_q = reward_batch + (1 - done_batch) * self.gamma * next_q
        
        # MSE Loss
        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        
        # Optimización
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        """Hard update de target network"""
        self.target_network.load_state_dict(self.q_network.state_dict())

# Crear agente
dqn_agent = DQNAgent(state_dim=4, action_dim=2)
print("Agente DQN creado exitosamente")

### 3.4 Función de Entrenamiento para DQN

In [ ]:
def train_dqn(env, agent, n_episodes=300, max_steps=500):
    """Entrena agente DQN"""
    rewards_history = []
    losses_history = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        episode_reward = 0
        episode_losses = []
        
        for step in range(max_steps):
            # Acción
            action = agent.get_action(state, training=True)
            
            # Ambiente
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Almacenar y entrenar
            agent.store_transition(state, action, reward, next_state, done)
            loss = agent.train_step()
            if loss is not None:
                episode_losses.append(loss)
            
            # Actualizar
            agent.steps += 1
            agent.update_epsilon()
            episode_reward += reward
            state = next_state
            
            if done:
                break
        
        # Target network update
        if episode % agent.target_update == 0:
            agent.update_target_network()
        
        rewards_history.append(episode_reward)
        avg_loss = np.mean(episode_losses) if episode_losses else 0
        losses_history.append(avg_loss)
        
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(rewards_history[-50:])
            print(f"Episode {episode + 1}/{n_episodes} | Avg Reward: {avg_reward:.1f} | Epsilon: {agent.epsilon:.3f}")
    
    return rewards_history, losses_history

print("Función de entrenamiento definida")

### 3.5 Entrenamiento en CartPole

In [ ]:
# Crear ambiente CartPole
env_cartpole = gym.make('CartPole-v1')
state_dim = env_cartpole.observation_space.shape[0]
action_dim = env_cartpole.action_space.n

print(f"CartPole-v1:")
print(f"  State dimension: {state_dim}")
print(f"  Action dimension: {action_dim}")
print(f"  Max steps: 500")
print(f"\nEntrenando DQN en CartPole... (esto toma ~1-2 minutos)")

# Crear y entrenar agente
dqn_agent = DQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    learning_rate=1e-3,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay=500,
    buffer_size=10000,
    batch_size=64,
    target_update=10
)

# Entrenar
dqn_rewards, dqn_losses = train_dqn(env_cartpole, dqn_agent, n_episodes=200, max_steps=500)

print(f"\nEntrenamiento completado!")
print(f"Reward promedio últimos 50 episodios: {np.mean(dqn_rewards[-50:]):.2f}")

### 3.6 Evaluación y Visualización de DQN

In [ ]:
# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Recompensas
ax1.plot(dqn_rewards, alpha=0.6, label='Reward', color='blue')
window = 10
if len(dqn_rewards) >= window:
    moving_avg = np.convolve(dqn_rewards, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(dqn_rewards)), moving_avg, 
            label=f'MA({window})', linewidth=2, color='darkblue')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Reward')
ax1.set_title('DQN Training Rewards (CartPole)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Losses
ax2.plot(dqn_losses, alpha=0.6, label='Loss', color='red')
if len(dqn_losses) >= window:
    moving_avg = np.convolve(dqn_losses, np.ones(window)/window, mode='valid')
    ax2.plot(range(window-1, len(dqn_losses)), moving_avg,
            label=f'MA({window})', linewidth=2, color='darkred')
ax2.set_xlabel('Episode')
ax2.set_ylabel('Loss')
ax2.set_title('DQN Training Loss (CartPole)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDQN entrenado exitosamente en CartPole!")

In [ ]:
# Evaluar agente entrenado
def evaluate_agent(env, agent, n_episodes=10, render=False):
    """Evalúa el agente sin exploración"""
    eval_rewards = []
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.get_action(state, training=False)  # Sin exploración
            state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward
        
        eval_rewards.append(episode_reward)
    
    return np.mean(eval_rewards), np.std(eval_rewards)

# Evaluar DQN
dqn_eval_mean, dqn_eval_std = evaluate_agent(env_cartpole, dqn_agent, n_episodes=20)
print(f"\nEvaluación DQN (20 episodios):")
print(f"  Reward promedio: {dqn_eval_mean:.2f}")
print(f"  Desv. estándar: {dqn_eval_std:.2f}")

## Sección 4: Double DQN

### Problema: Sobreestimación de Q-values

En DQN estándar:
$$Q_{\text{target}} = r + \gamma \max_{a'} Q_{\text{target}}(s', a')$$

El problema es que si todas las acciones son sobrestimadas, el máximo también lo será.

### Solución: Double DQN

Usar la red online para **seleccionar** y la red target para **evaluar**:
$$Q_{\text{target}} = r + \gamma Q_{\text{target}}(s', \arg\max_{a'} Q_{\text{online}}(s', a'))$$

Paper: "Deep Reinforcement Learning with Double Q-learning" (van Hasselt et al., 2015)

In [ ]:
class DoubleDQNAgent:
    """Agente Double DQN - Reduce sobreestimación"""
    def __init__(self, state_dim, action_dim,
                 learning_rate=1e-3,
                 gamma=0.99,
                 epsilon_start=1.0,
                 epsilon_end=0.01,
                 epsilon_decay=500,
                 buffer_size=10000,
                 batch_size=64,
                 target_update=10):
        
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update = target_update
        self.device = device
        
        # Redes (igual que DQN)
        self.q_network = DQN(state_dim, action_dim).to(self.device)
        self.target_network = DQN(state_dim, action_dim).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.replay_buffer = ReplayBuffer(buffer_size)
        self.steps = 0
    
    def get_action(self, state, training=True):
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_network(state_tensor)
            return q_values.argmax(dim=1).item()
    
    def update_epsilon(self):
        self.epsilon = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
                       np.exp(-self.steps / self.epsilon_decay)
    
    def store_transition(self, state, action, reward, next_state, done):
        self.replay_buffer.push(state, action, reward, next_state, done)
    
    def train_step(self):
        """Double DQN training step - Diferencia en el cálculo de targets"""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        transitions = self.replay_buffer.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.FloatTensor(np.array(batch.state)).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch.next_state)).to(self.device)
        done_batch = torch.FloatTensor(batch.done).to(self.device)
        
        current_q = self.q_network(state_batch).gather(1, action_batch)
        
        with torch.no_grad():
            # Double DQN: Diferencia clave aquí
            # 1. Seleccionar acciones con online network
            next_actions = self.q_network(next_state_batch).argmax(1, keepdim=True)
            # 2. Evaluar con target network
            next_q = self.target_network(next_state_batch).gather(1, next_actions).squeeze()
            target_q = reward_batch + (1 - done_batch) * self.gamma * next_q
        
        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

print("Clase DoubleDQNAgent definida")

In [ ]:
# Entrenar Double DQN
print("Entrenando Double DQN en CartPole...")

ddqn_agent = DoubleDQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    learning_rate=1e-3,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay=500,
    buffer_size=10000,
    batch_size=64,
    target_update=10
)

ddqn_rewards, ddqn_losses = train_dqn(env_cartpole, ddqn_agent, n_episodes=200, max_steps=500)

print(f"\nDouble DQN entrenado!")
print(f"Reward promedio últimos 50 episodios: {np.mean(ddqn_rewards[-50:]):.2f}")

In [ ]:
# Comparar DQN vs Double DQN
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

window = 10

# DQN
dqn_ma = np.convolve(dqn_rewards, np.ones(window)/window, mode='valid')
ax1.plot(range(window-1, len(dqn_rewards)), dqn_ma, label='DQN', linewidth=2, color='blue')

# Double DQN
ddqn_ma = np.convolve(ddqn_rewards, np.ones(window)/window, mode='valid')
ax1.plot(range(window-1, len(ddqn_rewards)), ddqn_ma, label='Double DQN', linewidth=2, color='green')

ax1.set_xlabel('Episode')
ax1.set_ylabel('Average Reward (MA-10)')
ax1.set_title('DQN vs Double DQN')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Losses
dqn_loss_ma = np.convolve(dqn_losses, np.ones(window)/window, mode='valid')
ddqn_loss_ma = np.convolve(ddqn_losses, np.ones(window)/window, mode='valid')

ax2.plot(range(window-1, len(dqn_losses)), dqn_loss_ma, label='DQN', linewidth=2, color='blue')
ax2.plot(range(window-1, len(ddqn_losses)), ddqn_loss_ma, label='Double DQN', linewidth=2, color='green')
ax2.set_xlabel('Episode')
ax2.set_ylabel('Loss (MA-10)')
ax2.set_title('DQN vs Double DQN - Training Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Evaluar
ddqn_eval_mean, ddqn_eval_std = evaluate_agent(env_cartpole, ddqn_agent, n_episodes=20)
print(f"\nEvaluación Double DQN (20 episodios):")
print(f"  Reward promedio: {ddqn_eval_mean:.2f}")
print(f"  Desv. estándar: {ddqn_eval_std:.2f}")

In [ ]:
# Análisis de diferencias
print("\n" + "="*50)
print("ANÁLISIS: DQN vs Double DQN")
print("="*50)

print("\nDQN Estándar:")
print(f"  Target = r + γ * max_a' Q_target(s', a')")
print(f"  Problema: Sobreestimación si todo está sobrestimado")

print("\nDouble DQN:")
print(f"  Target = r + γ * Q_target(s', argmax_a' Q_online(s', a'))")
print(f"  Ventaja: Desacopla selección de evaluación")

print("\nResultados en CartPole-v1:")
print(f"  DQN:        {dqn_eval_mean:.2f} ± {dqn_eval_std:.2f}")
print(f"  Double DQN: {ddqn_eval_mean:.2f} ± {ddqn_eval_std:.2f}")

## Sección 5: Dueling DQN

### Arquitectura Dueling

Separa la estimación de Q-values en dos streams:
- **Value stream V(s)**: Valor de estar en el estado s
- **Advantage stream A(s,a)**: Ventaja de tomar la acción a respecto al promedio

$$Q(s,a) = V(s) + \left(A(s,a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s,a')\right)$$

### Ventajas
1. Aprende qué estados son valiosos sin aprender el efecto de cada acción
2. Mejor generalización en espacios de acción grandes
3. Útil cuando muchas acciones tienen efecto similar

Paper: "Dueling Network Architectures for Deep Reinforcement Learning" (Wang et al., 2016)

In [ ]:
class DuelingDQN(nn.Module):
    """Red Dueling DQN - Separa Value y Advantage"""
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(DuelingDQN, self).__init__()
        
        # Feature extraction (compartido)
        self.feature = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Value stream: V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)  # Un valor
        )
        
        # Advantage stream: A(s,a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, action_dim)  # Ventaja por acción
        )
        
        self.action_dim = action_dim
    
    def forward(self, x):
        # Features compartidas
        features = self.feature(x)
        
        # Value stream
        value = self.value_stream(features)  # [batch, 1]
        
        # Advantage stream
        advantage = self.advantage_stream(features)  # [batch, action_dim]
        
        # Aggregation: Q(s,a) = V(s) + (A(s,a) - mean_a(A(s,a)))
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        
        return q_values
    
    def get_value_and_advantage(self, x):
        """Retorna V(s) y A(s,a) por separado para análisis"""
        features = self.feature(x)
        value = self.value_stream(features)
        advantage = self.advantage_stream(features)
        return value, advantage

# Prueba
dueling_net = DuelingDQN(state_dim=4, action_dim=2).to(device)
print("Arquitectura Dueling DQN:")
print(dueling_net)

# Forward pass
dummy_state = torch.randn(1, 4).to(device)
q_vals = dueling_net(dummy_state)
v, a = dueling_net.get_value_and_advantage(dummy_state)
print(f"\nValue V(s): {v.item():.3f}")
print(f"Advantage A(s,a): {a.cpu().detach().numpy()[0]}")
print(f"Q-values Q(s,a): {q_vals.cpu().detach().numpy()[0]}")

In [ ]:
class DuelingDQNAgent:
    """Agente Dueling DQN"""
    def __init__(self, state_dim, action_dim,
                 learning_rate=1e-3,
                 gamma=0.99,
                 epsilon_start=1.0,
                 epsilon_end=0.01,
                 epsilon_decay=500,
                 buffer_size=10000,
                 batch_size=64,
                 target_update=10):
        
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update = target_update
        self.device = device
        
        # Usar arquitectura Dueling
        self.q_network = DuelingDQN(state_dim, action_dim).to(self.device)
        self.target_network = DuelingDQN(state_dim, action_dim).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.replay_buffer = ReplayBuffer(buffer_size)
        self.steps = 0
    
    def get_action(self, state, training=True):
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_network(state_tensor)
            return q_values.argmax(dim=1).item()
    
    def update_epsilon(self):
        self.epsilon = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
                       np.exp(-self.steps / self.epsilon_decay)
    
    def store_transition(self, state, action, reward, next_state, done):
        self.replay_buffer.push(state, action, reward, next_state, done)
    
    def train_step(self):
        """Training con arquitectura Dueling"""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        transitions = self.replay_buffer.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.FloatTensor(np.array(batch.state)).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch.next_state)).to(self.device)
        done_batch = torch.FloatTensor(batch.done).to(self.device)
        
        # Forward pass - arquitectura Dueling calcula Q automáticamente
        current_q = self.q_network(state_batch).gather(1, action_batch)
        
        with torch.no_grad():
            next_q = self.target_network(next_state_batch).max(1)[0]
            target_q = reward_batch + (1 - done_batch) * self.gamma * next_q
        
        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())
    
    def analyze_state(self, state):
        """Analiza V(s) y A(s,a) para un estado"""
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            value, advantage = self.q_network.get_value_and_advantage(state_tensor)
            q_values = self.q_network(state_tensor)
            
            return {
                'value': value.cpu().numpy()[0, 0],
                'advantage': advantage.cpu().numpy()[0],
                'q_values': q_values.cpu().numpy()[0]
            }

print("Clase DuelingDQNAgent definida")

In [ ]:
# Entrenar Dueling DQN
print("Entrenando Dueling DQN en CartPole...")

dueling_agent = DuelingDQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    learning_rate=1e-3,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay=500,
    buffer_size=10000,
    batch_size=64,
    target_update=10
)

dueling_rewards, dueling_losses = train_dqn(env_cartpole, dueling_agent, n_episodes=200, max_steps=500)

print(f"\nDueling DQN entrenado!")
print(f"Reward promedio últimos 50 episodios: {np.mean(dueling_rewards[-50:]):.2f}")

In [ ]:
# Analizar Value y Advantage de un estado
test_state, _ = env_cartpole.reset()
analysis = dueling_agent.analyze_state(test_state)

print("\nAnálisis de arquitectura Dueling DQN:")
print(f"Estado inicial: {test_state}")
print(f"\nValue V(s): {analysis['value']:.4f}")
print(f"Advantage A(s,a): {analysis['advantage']}")
print(f"Q-values Q(s,a): {analysis['q_values']}")
print(f"\nMejor acción: {np.argmax(analysis['q_values'])}")

In [ ]:
# Comparar los 3 algoritmos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

window = 10
algorithms = ['DQN', 'Double DQN', 'Dueling DQN']
rewards_list = [dqn_rewards, ddqn_rewards, dueling_rewards]
colors = ['blue', 'green', 'red']

for i, (ax, algo, rewards, color) in enumerate(zip(axes, algorithms, rewards_list, colors)):
    ma = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), ma, linewidth=2, color=color)
    ax.set_xlabel('Episode')
    ax.set_ylabel('Average Reward (MA-10)')
    ax.set_title(f'{algo}')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0, 500])

plt.tight_layout()
plt.show()

# Evaluar
dueling_eval_mean, dueling_eval_std = evaluate_agent(env_cartpole, dueling_agent, n_episodes=20)
print(f"\nEvaluación Dueling DQN (20 episodios):")
print(f"  Reward promedio: {dueling_eval_mean:.2f}")
print(f"  Desv. estándar: {dueling_eval_std:.2f}")

## Sección 6: Comparación de los 3 Algoritmos

Análisis detallado de ventajas y desventajas

In [ ]:
# Comparación visual lado a lado
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

window = 10

# Recompensas
dqn_ma = np.convolve(dqn_rewards, np.ones(window)/window, mode='valid')
ddqn_ma = np.convolve(ddqn_rewards, np.ones(window)/window, mode='valid')
dueling_ma = np.convolve(dueling_rewards, np.ones(window)/window, mode='valid')

ax1.plot(range(window-1, len(dqn_rewards)), dqn_ma, label='DQN', linewidth=2.5, color='blue')
ax1.plot(range(window-1, len(ddqn_rewards)), ddqn_ma, label='Double DQN', linewidth=2.5, color='green')
ax1.plot(range(window-1, len(dueling_rewards)), dueling_ma, label='Dueling DQN', linewidth=2.5, color='red')
ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Average Reward (MA-10)', fontsize=12)
ax1.set_title('Comparación de Algoritmos - Recompensas', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Losses
dqn_loss_ma = np.convolve(dqn_losses, np.ones(window)/window, mode='valid')
ddqn_loss_ma = np.convolve(ddqn_losses, np.ones(window)/window, mode='valid')
dueling_loss_ma = np.convolve(dueling_losses, np.ones(window)/window, mode='valid')

ax2.plot(range(window-1, len(dqn_losses)), dqn_loss_ma, label='DQN', linewidth=2.5, color='blue')
ax2.plot(range(window-1, len(ddqn_losses)), ddqn_loss_ma, label='Double DQN', linewidth=2.5, color='green')
ax2.plot(range(window-1, len(dueling_losses)), dueling_loss_ma, label='Dueling DQN', linewidth=2.5, color='red')
ax2.set_xlabel('Episode', fontsize=12)
ax2.set_ylabel('Loss (MA-10)', fontsize=12)
ax2.set_title('Comparación de Algoritmos - Training Loss', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Tabla de comparación
import pandas as pd

comparison_data = {
    'Algoritmo': ['DQN', 'Double DQN', 'Dueling DQN'],
    'Reward Final (Train)': [
        f"{np.mean(dqn_rewards[-50:]):.2f}",
        f"{np.mean(ddqn_rewards[-50:]):.2f}",
        f"{np.mean(dueling_rewards[-50:]):.2f}"
    ],
    'Reward Eval': [
        f"{dqn_eval_mean:.2f} ± {dqn_eval_std:.2f}",
        f"{ddqn_eval_mean:.2f} ± {ddqn_eval_std:.2f}",
        f"{dueling_eval_mean:.2f} ± {dueling_eval_std:.2f}"
    ],
    'Complejidad': ['Baja', 'Baja', 'Media'],
    'Estabilidad': ['Buena', 'Mejor', 'Muy Buena']
}

df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("COMPARACIÓN CUANTITATIVA DE ALGORITMOS")
print("="*80)
print(df.to_string(index=False))
print("="*80)

### Análisis Cualitativo

| Aspecto | DQN | Double DQN | Dueling DQN |
|--------|-----|-----------|-------------|
| **Ecuación Update** | r + γ max Q_t(s',a') | r + γ Q_t(s',argmax Q(s',a')) | Arquitectura V+A |
| **Problema Principal** | Sobreestimación | Reducida | Separación V/A |
| **Convergencia** | Lenta | Más rápida | Mejor generalización |
| **Complejidad Código** | Simple | Muy simple | Media |
| **Uso de Memoria** | Normal | Normal | Ligeramente mayor |
| **Mejoras Compatibles** | PER, Dueling | PER, Dueling | Double, PER |
| **Caso de Uso Ideal** | Inicio/Simple | Espacios acción pequeños | Espacios acción grandes |
| **Paper** | Mnih et al. (2015) | van Hasselt et al. (2015) | Wang et al. (2016) |

In [ ]:
# Resumen de claves
print("\n" + "="*80)
print("RESUMEN TÉCNICO: DIFERENCIAS CLAVE")
print("="*80)

summary = """
1. DQN (Mnih et al., 2015)
   - Usa Experience Replay para decorrelacionar datos
   - Target Network separada para estabilidad
   - Update: Q ← r + γ max Q_target(s')
   - Problema: Max de máximos sobrestima valores

2. Double DQN (van Hasselt et al., 2015)
   - Pequeño cambio, gran impacto
   - Selecciona acción con Q-online, evalúa con Q-target
   - Update: Q ← r + γ Q_target(s', argmax_a Q_online(s',a))
   - Reduces sesgo de sobreestimación

3. Dueling DQN (Wang et al., 2016)
   - Arquitectura dual: Value stream + Advantage stream
   - V(s): Valor intrínseco del estado
   - A(s,a): Ventaja relativa de cada acción
   - Q(s,a) = V(s) + (A(s,a) - mean_a A(s,a))
   - Mejor para espacios de acción grandes
"""

print(summary)
print("="*80)

## Sección 7: Ejercicios Prácticos

Tareas para consolidar el aprendizaje

### Ejercicio 1: Implementar DQN con Soft Update

Modifica el DQNAgent para incluir soft update de la target network.

Soft update: θ_target ← τ * θ_online + (1-τ) * θ_target

Ventajas: Actualizaciones más suaves, mejor estabilidad

In [ ]:
class SoftUpdateDQNAgent(DQNAgent):
    """DQN con Soft Update (TAU)"""
    
    def __init__(self, state_dim, action_dim, tau=0.01, **kwargs):
        """tau: factor de soft update (0.01 típico)"""
        super().__init__(state_dim, action_dim, **kwargs)
        self.tau = tau
    
    def soft_update_target_network(self):
        """Soft update de target network"""
        for target_param, param in zip(
            self.target_network.parameters(),
            self.q_network.parameters()
        ):
            target_param.data.copy_(
                self.tau * param.data + (1.0 - self.tau) * target_param.data
            )
    
    # El entrenamiento es igual, solo cambia el update

# Prueba
print("Ejercicio 1: Soft Update implementado")
print("TODO: Entrena un agente con soft update (tau=0.01) y compara convergencia")

### Ejercicio 2: Hiperparámetros Óptimos

Experimenta con diferentes valores de:
- Learning rate (1e-4, 5e-4, 1e-3, 5e-3)
- Gamma (0.95, 0.99, 0.999)
- Epsilon decay (100, 500, 1000)
- Buffer size (5000, 10000, 50000)

¿Cuál combinación da mejor rendimiento?

In [ ]:
# Plantilla para grid search

hyperparams = {
    'learning_rate': [1e-3],  # Añade más valores
    'gamma': [0.99],           # Añade más valores
    'epsilon_decay': [500],    # Añade más valores
    'batch_size': [64]         # Añade más valores
}

# TODO: Implementa búsqueda exhaustiva o Bayesiana
# Consejos:
# 1. Comienza con 2-3 valores por parámetro
# 2. Usa reward promedio últimos 50 episodios como métrica
# 3. Ejecuta múltiples seeds para varianza

print("Ejercicio 2: Búsqueda de Hiperparámetros")
print("TODO: Implementa grid search sobre los hiperparámetros")

### Ejercicio 3: Entrenar en LunarLander

LunarLander-v2 es más complejo que CartPole.
- Estado: 8 dimensiones
- Acciones: 4 discretas
- Recompensa: -200 a 200+

Requisitos:
- Network más grande (256 hidden)
- Buffer más grande (50000)
- Más episodios de entrenamiento (500+)
- Ajustar learning rate

In [ ]:
# Plantilla para LunarLander

def train_on_lunar_lander():
    """Entrena los 3 algoritmos en LunarLander-v2"""
    env = gym.make('LunarLander-v2')
    
    state_dim = env.observation_space.shape[0]  # 8
    action_dim = env.action_space.n              # 4
    
    print(f"LunarLander-v2: state_dim={state_dim}, action_dim={action_dim}")
    
    # Configuración para LunarLander (más exigente)
    config = {
        'learning_rate': 5e-4,      # Más bajo para estabilidad
        'gamma': 0.99,
        'epsilon_start': 1.0,
        'epsilon_end': 0.01,
        'epsilon_decay': 1000,      # Más largo
        'buffer_size': 50000,       # Más grande
        'batch_size': 128,          # Más grande
        'target_update': 5          # Más frecuente
    }
    
    # TODO: Entrena aquí
    # Objetivo: Conseguir reward > 200 (considérate experto con >250)
    
    env.close()
    return None

print("Ejercicio 3: LunarLander-v2")
print("TODO: Implementa entrenamiento en ambiente más complejo")
# train_on_lunar_lander()  # Descomenta para ejecutar

### Ejercicio 4: Análisis de Value vs Advantage

Visualiza cómo evolucionan V(s) y A(s,a) durante el entrenamiento

In [ ]:
# Muestrear estados y analizar Value/Advantage
values_collected = []
advantages_collected = []

# Episodios de análisis
for ep in range(5):
    state, _ = env_cartpole.reset()
    done = False
    
    while not done:
        analysis = dueling_agent.analyze_state(state)
        values_collected.append(analysis['value'])
        advantages_collected.extend(analysis['advantage'])
        
        action = dueling_agent.get_action(state, training=False)
        state, _, terminated, truncated, _ = env_cartpole.step(action)
        done = terminated or truncated

# Visualizar distribuciones
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(values_collected, bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[0].set_xlabel('V(s)')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Values V(s)')
axes[0].grid(True, alpha=0.3)

axes[1].hist(advantages_collected, bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1].set_xlabel('A(s,a)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Advantages A(s,a)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEstadísticas Value:")
print(f"  Media: {np.mean(values_collected):.3f}")
print(f"  Desv: {np.std(values_collected):.3f}")
print(f"\nEstadísticas Advantage:")
print(f"  Media: {np.mean(advantages_collected):.3f}")
print(f"  Desv: {np.std(advantages_collected):.3f}")

### Ejercicio 5: Guardar y Cargar Modelos

Implementa persistencia de modelos

In [ ]:
def save_agent(agent, path):
    """Guarda el agente DQN"""
    checkpoint = {
        'q_network': agent.q_network.state_dict(),
        'target_network': agent.target_network.state_dict(),
        'optimizer': agent.optimizer.state_dict(),
        'epsilon': agent.epsilon,
        'steps': agent.steps
    }
    torch.save(checkpoint, path)
    print(f"Modelo guardado en {path}")

def load_agent(agent, path):
    """Carga el agente DQN"""
    checkpoint = torch.load(path, map_location=device)
    agent.q_network.load_state_dict(checkpoint['q_network'])
    agent.target_network.load_state_dict(checkpoint['target_network'])
    agent.optimizer.load_state_dict(checkpoint['optimizer'])
    agent.epsilon = checkpoint['epsilon']
    agent.steps = checkpoint['steps']
    print(f"Modelo cargado desde {path}")

# Ejemplo
save_agent(dqn_agent, '/tmp/dqn_model.pth')
load_agent(dqn_agent, '/tmp/dqn_model.pth')

print("\nModelos guardados y cargados exitosamente")

## Sección 8: Conclusiones

### Resumen de lo Aprendido

1. **Deep Q-Network (DQN)** es el fundamento de Deep RL
   - Combina Q-Learning con redes neuronales
   - Experience Replay rompe correlación temporal
   - Target Network proporciona estabilidad

2. **Double DQN** mejora DQN reduciendo sobreestimación
   - Cambio pequeño: desacopla selección de evaluación
   - Mayor estabilidad en entrenamiento
   - Mejor rendimiento en muchos ambientes

3. **Dueling DQN** utiliza arquitectura dual
   - Separa Value (qué tan bueno es estado) de Advantage (qué tan buena es acción)
   - Mejor generalización en espacios de acción grandes
   - Se combina perfectamente con Double DQN

4. **Progresión de mejoras**
   - DQN (2015): Fundacional
   - Double DQN (2015): Muy poco código extra, gran impacto
   - Dueling DQN (2016): Mejor arquitectura
   - Rainbow (2017): Combina TODO + PER + Noisy

### Próximos Pasos

1. **Prioritized Experience Replay (PER)**
   - Muestrea transiciones importantes (TD error alto)
   - Acelera aprendizaje significativamente

2. **Noisy Networks**
   - Exploración paramétrica en lugar de ε-greedy
   - Mejor exploración coordenada

3. **Distributional RL**
   - C51: Aprender distribución de Q, no solo esperanza
   - QR-DQN: Quantile Regression

4. **Policy Gradient Methods**
   - Actor-Critic
   - PPO (Proximal Policy Optimization)
   - TRPO (Trust Region Policy Optimization)

5. **Multi-Agent RL**
   - QMIX: Mezcla cooperativa de Q-values
   - MADDPG: Multi-Agent Deep Deterministic Policy Gradient

In [ ]:
# Resumen final
print("\n" + "="*80)
print("CONCLUSIÓN: DEEP REINFORCEMENT LEARNING CON DQN")
print("="*80)

summary_stats = f"""
RESULTADOS EN CARTPOLE-V1 (200 episodios de entrenamiento):

  DQN:
    - Reward Training (MA-50): {np.mean(dqn_rewards[-50:]):.2f}
    - Reward Evaluación: {dqn_eval_mean:.2f} ± {dqn_eval_std:.2f}
    - Complejidad: Baja

  Double DQN:
    - Reward Training (MA-50): {np.mean(ddqn_rewards[-50:]):.2f}
    - Reward Evaluación: {ddqn_eval_mean:.2f} ± {ddqn_eval_std:.2f}
    - Complejidad: Muy baja (1 línea diferencia)

  Dueling DQN:
    - Reward Training (MA-50): {np.mean(dueling_rewards[-50:]):.2f}
    - Reward Evaluación: {dueling_eval_mean:.2f} ± {dueling_eval_std:.2f}
    - Complejidad: Media

LECCIONES CLAVE:

1. DQN es accesible pero fundamental
   - Implementación clara de conceptos clave
   - Buenos resultados con tuning mínimo

2. Mejoras incrementales importan mucho
   - Double DQN: cambio mínimo, mejora significativa
   - Dueling: arquitectura mejor, más insumos

3. Tuning es crítico en Deep RL
   - Learning rate, gamma, epsilon decay
   - Tamaño buffer y batch size
   - Hidden dimensions y target update

4. Validación rigurosa necesaria
   - Múltiples seeds (mínimo 3)
   - Evaluation sin exploración
   - Media móvil para noisy rewards

RECOMENDACIONES DE USO:

  Usa DQN si:
    - Es tu primer algoritmo
    - Necesitas baseline simple
    - Ambiente pequeño, acción discreta

  Usa Double DQN si:
    - Sospechas sobreestimación
    - Entrenamiento inestable
    - Poco overhead, mucho beneficio

  Usa Dueling DQN si:
    - Muchas acciones disponibles
    - Espacio de acción discreto grande
    - Necesitas mejor generalización

  Usa Rainbow si:
    - Quieres estado del arte
    - Tienes budget computacional
    - Performance es crítica
"""

print(summary_stats)
print("="*80)
print("\n¡Felicidades! Has completado el tutorial de Deep RL/DQN.")
print("Ahora eres capaz de:")
print("  - Implementar DQN desde cero")
print("  - Entender Double DQN y Dueling DQN")
print("  - Entrenar en ambientes complejos")
print("  - Analizar y debuggear algoritmos")
print("\nPróximos desafíos: Explora Policy Gradient, Actor-Critic y técnicas avanzadas.")

In [ ]:
# Cerrar ambiente
env_cartpole.close()
print("\nTutorial completado exitosamente!")